# Branching Ratios for V-type Sink Model

From §5.3.2 in my thesis:

<blockquote>
Firstly the matrix elements can be factored such that the dependence on the
$m_F$ and $m_F'$ quantum numbers is entirely within a Wigner 3-j factor, via
the expression

$$
\begin{align}
  \langle F m_F | d_q | F' m'_F \rangle &= 
  \langle F  \| e \mathbf{r} \| F' \rangle \nonumber \\
  &\times (-1)^{F' - 1 + m_F} \sqrt{2F + 1}
  \begin{pmatrix}
    F' & 1 & F \\
    m_F' & q & -m_F 
  \end{pmatrix}
  \tag{1}
\end{align}
$$

where $\langle F  \| e \mathbf{r} \| F' \rangle$ is now the reduced
hyperfine dipole matrix element.

$$
\begin{align}
  \langle F  \| e \mathbf{r} \| F' \rangle =
  &\langle J  \| e \mathbf{r} \| J' \rangle \nonumber \\
  &\times (-1)^{F' + J + 1 + I}
  \sqrt{(2F' + 1)(2J + 1)}
        \begin{Bmatrix}
    J & J' & 1 \\
    F' & F & I 
  \end{Bmatrix}
  \tag{2}
\end{align}
$$
</blockquote>

I wrote the method `ang_mom.calc_clebsch_hf` in the package to calculate the hyperfine factors (i.e. $c$ where $\langle F m_F | d_q | F' m'_F \rangle = \langle J  \| e \mathbf{r} \| J' \rangle \times c$). These are the same listed in tables 9-20 in Steck's Rubidium 87 data.

The method is documented below:

In [127]:
from opticalbloch import ang_mom
from math import sqrt

help(ang_mom.calc_clebsch_hf)

Help on function calc_clebsch_hf in module opticalbloch.ang_mom:

calc_clebsch_hf(J_a, I_a, F_a, mF_a, J_b, I_b, F_b, mF_b, q)
    Clebsch-Gordan coefficient for the hyperfine transition dipole matrix
    element



## Factors for $5S_{1/2} \rightarrow 5P_{1/2}$

Let's try a randomly picked example. Steck's Table 16, gives the factor for $|{F=2, m_F=0}\rangle \rightarrow | F'=1, m'_F=0 \rangle $ as $1/\sqrt{3}$. (In the method I use `a` to index the lower state and `b` to index the upper state.)

In [129]:
hf_factor = ang_mom.calc_clebsch_hf(J_a=1/2.,
                                    I_a=3/2., # Rubidium 87 nuclear spin
                                    F_a=2., 
                                    mF_a=0., 
                                    J_b=1/2., 
                                    I_b=3/2., 
                                    F_b=1., 
                                    mF_b=0., 
                                    q=0.)

In [132]:
print(hf_factor)
print(1/sqrt(3.))

0.57735026919
0.57735026919


Another example. Steck's Table 15, gives the factor for $|{F=2, m_F=-1}\rangle \rightarrow | F'=2, m_F'=0 \rangle $ as $1/\sqrt{4}$.

In [26]:
hf_factor = ang_mom.calc_clebsch_hf(J_a=1/2.,
                                    I_a=3/2., # Rubidium 87 nuclear spin
                                    F_a=2., 
                                    mF_a=-1., 
                                    J_b=1/2., 
                                    I_b=3/2., 
                                    F_b=2., 
                                    mF_b=0., 
                                    q=-1.)

print(hf_factor)
print(1/sqrt(4.))

0.5
0.5


Feel free to check further factors!

Now, as my $I$s and $J$s are always the same, I'll make a lambda function that only requires the parameters: `F_a, mF_a, F_b, mF_b, q`, just to cut clutter.

In [136]:
cg_hf = lambda F_a, mF_a, F_b, mF_b, q: ang_mom.calc_clebsch_hf(1/2., 3/2., F_a, 
                                                                mF_a, 1/2., 3/2., 
                                                                F_b, mF_b, q)

Same two checks as above:

In [139]:
print(cg_hf(2., 0., 1., 0, 0.))
print(cg_hf(2., -1., 2., 0, -1.))

0.57735026919
0.5


Now, let's write a function to take any upper $F', m'_F$ level and sum up all the factors coupled to it:

In [140]:
def sum_to_upper(F_b, mF_b, log=False):
    sum_cg_hf = 0.
    for q in [-1., 0., 1]:
        for F_a in [1., 2.]:
            for mF_a in [-2., -1., 0., 1., 2.]:
                c = cg_hf(F_a, mF_a, F_b, mF_b, q)
                if log == True and c != 0:
                    print q, F_a, mF_a, cg_hf(F_a, mF_a, F_b, mF_b, q)
                sum_cg_hf += cg_hf(F_a, mF_a, F_b, mF_b, q)**2
    return sum_cg_hf

Eqn 40 in Steck tells us that 

> the matrix elements that couple to any single excited state sublevel $|F' m'_F \rangle$ add up to a factor that is independent of the particular sublevel chosen.

and that the result should be $(2J + 1)/(2J' + 1)$.

If I pick say, $|F'=1 m'_F=0 \rangle$, I expect there to be 5 non-zero couplings (from the selection rules)

In [148]:
print("Sum F_b=1., mF_b=0.: {0}".format(sum_to_upper(F_b=1., mF_b = 0., log=True)))  

-1.0 1.0 -1.0 -0.288675134595
-1.0 2.0 -1.0 0.5
0.0 2.0 0.0 0.57735026919
1 1.0 1.0 0.288675134595
1 2.0 1.0 0.5
Sum F_b=1., mF_b=0.: 1.0


and sure enough there are, and they sum to $(2J + 1)/(2J' + 1) = 1$. Good. Now we can check this is the case for _all_ upper sublevels.

In [149]:
for F_b in [1., 2.]:
    for mF_b in [-2., -1., 0., 1., 2.]:
        print("Sum F_b={0}, mF_b={1}: {2}".format(F_b, mF_b, sum_to_upper(F_b, mF_b)))  

Sum F_b=1.0, mF_b=-2.0: 0.0
Sum F_b=1.0, mF_b=-1.0: 1.0
Sum F_b=1.0, mF_b=0.0: 1.0
Sum F_b=1.0, mF_b=1.0: 1.0
Sum F_b=1.0, mF_b=2.0: 0.0
Sum F_b=2.0, mF_b=-2.0: 1.0
Sum F_b=2.0, mF_b=-1.0: 1.0
Sum F_b=2.0, mF_b=0.0: 1.0
Sum F_b=2.0, mF_b=1.0: 1.0
Sum F_b=2.0, mF_b=2.0: 1.0


We see the symmetry. So this verifies Steck Eqn 40 for our case.

### Summing the branching ratios to a given upper F', m_F'

Eqn 41 in Steck gives us the $S_{FF'}$ strength factors,

> summing the matrix elements from a single ground-state sublevel to the levels in a particular F′ energy level

which is 

> independent of the particular ground state sublevel chosen.

For our purpose of having a single upper state representing the two $F'$ levels with the correct branching to two $F$ lower levels, **we want the converse**: the sum of the matrix elements **to** a single upper-state sublevel **from** all sublevels in a given $F$ lower level.

Let's write a function to get that:

In [151]:
def sum_to_upper_from_F(F_a, F_b, mF_b, log=False):
    sum_cg_hf = 0.
    for q in [-1., 0., 1]:
        for mF_a in [-2., -1., 0., 1., 2.]:
            c = cg_hf(F_a, mF_a, F_b, mF_b, q)
            if log == True and c != 0:
                print q, F_a, mF_a, cg_hf(F_a, mF_a, F_b, mF_b, q)
            sum_cg_hf += cg_hf(F_a, mF_a, F_b, mF_b, q)**2
    return sum_cg_hf

We'll test it for $F = 1 \rightarrow F'=1, m'_F=0$.

In [154]:
print("Sum F_a=1., F_b=1., mF_b=0.: {0}".format(sum_to_upper_from_F(F_a=1., 
                                                                    F_b=1., 
                                                                    mF_b = 0.,
                                                                    log=True)))  

-1.0 1.0 -1.0 -0.288675134595
1 1.0 1.0 0.288675134595
Sum F_a=1., F_b=1., mF_b=0.: 0.166666666667


which is 1/6.

In [156]:
print(1/6.)

0.166666666667


Now let's try it for each of the upper $F'=1, m'_F$ states from $F=1$:

In [158]:
F_a=1.
F_b=1.

for mF_b in [-2., -1., 0., 1., 2.]:
    print("Sum F_a={0}, F_b={1}, mF_b={2}: {3}".format(F_a, F_b, mF_b, 
                                                       sum_to_upper_from_F(F_a, F_b, mF_b)))  

Sum F_a=1.0, F_b=1.0, mF_b=-2.0: 0.0
Sum F_a=1.0, F_b=1.0, mF_b=-1.0: 0.166666666667
Sum F_a=1.0, F_b=1.0, mF_b=0.0: 0.166666666667
Sum F_a=1.0, F_b=1.0, mF_b=1.0: 0.166666666667
Sum F_a=1.0, F_b=1.0, mF_b=2.0: 0.0


And we see a similar symmetry. **The coupling to a given upper state is independent of the particular upper state sublevel chosen.**

Let's try for $F = 1 \rightarrow F' = 2$.

In [159]:
F_a=1.
F_b=2.

for mF_b in [-2., -1., 0., 1., 2.]:
    print("Sum F_a={0}, F_b={1}, mF_b={2}: {3}".format(F_a, F_b, mF_b, 
                                                       sum_to_upper_from_F(F_a, F_b, mF_b))) 

Sum F_a=1.0, F_b=2.0, mF_b=-2.0: 0.5
Sum F_a=1.0, F_b=2.0, mF_b=-1.0: 0.5
Sum F_a=1.0, F_b=2.0, mF_b=0.0: 0.5
Sum F_a=1.0, F_b=2.0, mF_b=1.0: 0.5
Sum F_a=1.0, F_b=2.0, mF_b=2.0: 0.5


And for completeness, $F = 2 \rightarrow F' = 1$...

In [162]:
F_a=2.
F_b=1.

for mF_b in [-2., -1., 0., 1., 2.]:
    print("Sum F_a={0}, F_b={1}, mF_b={2}: {3}".format(F_a, F_b, mF_b, 
                                                       sum_to_upper_from_F(F_a, F_b, mF_b))) 

Sum F_a=2.0, F_b=1.0, mF_b=-2.0: 0.0
Sum F_a=2.0, F_b=1.0, mF_b=-1.0: 0.833333333333
Sum F_a=2.0, F_b=1.0, mF_b=0.0: 0.833333333333
Sum F_a=2.0, F_b=1.0, mF_b=1.0: 0.833333333333
Sum F_a=2.0, F_b=1.0, mF_b=2.0: 0.0


and $F = 2 \rightarrow F' = 2$.

In [163]:
F_a=2.
F_b=2.

for mF_b in [-2., -1., 0., 1., 2.]:
    print("Sum F_a={0}, F_b={1}, mF_b={2}: {3}".format(F_a, F_b, mF_b, 
                                                       sum_to_upper_from_F(F_a, F_b, mF_b))) 

Sum F_a=2.0, F_b=2.0, mF_b=-2.0: 0.5
Sum F_a=2.0, F_b=2.0, mF_b=-1.0: 0.5
Sum F_a=2.0, F_b=2.0, mF_b=0.0: 0.5
Sum F_a=2.0, F_b=2.0, mF_b=1.0: 0.5
Sum F_a=2.0, F_b=2.0, mF_b=2.0: 0.5


Now notice that if we call these branching factors $T_{FF'}$, we have $T_{FF'} = S_{F'F}$ (i.e. compare the results above with Steck Table 8.)

| |  |
| --- | --- |
| $T_{11}$ = 1/6 | $T_{12}$ = 1/2 |
| $T_{21}$ = 5/6 | $T_{22}$ = 1/2 |

The reason for this will be in changing Eqn 41 for our purpose (I suspect changing the sign of $q$ is necessary?)

Now what we want is the sum of the branching ratios from $F = 1$ which is 

$$
(T_{11} + T_{12})/(2J' + 1) = (1/6 + 1/2)/2 = 1/3
$$

and the sum of the branching ratios from $F = 2$ which is 

$$
(T_{21} + T_{22})/(2J' + 1) = (5/6 + 1/2) = 2/3
$$

These sum to 1, which we would expect form branching ratios.

We can make the same checks with $5S_{1/2} \rightarrow 5P_{3/2}$, remembering that we will then have a different value for $1/(2J' + 1) = 1/4$.
                                                                                                                                